In [1]:
import pickle
import time
import math
import random

class MaximumCliqueSA:
    def __init__(self, nam_file):
        try:
            with open(nam_file, 'rb') as f:
                self.codes_int = pickle.load(f)
        except FileNotFoundError:
            print(f"Error: {nam_file} not found.")
            self.codes_int = []
            
        # Nodes are assumed to be 1-indexed based on your provided snippet
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        """Checks edge existence using the bitmask logic from your data format."""
        if u == v: return False
        return ((self.codes_int[u] >> v) & 1) == 0

    def get_node_conflicts(self, node, S):
        """Returns how many nodes in set S are NOT connected to 'node'."""
        return sum(1 for member in S if not self.are_neighbors(node, member))

    def calculate_total_conflicts(self, S):
        """Calculates total non-edges within the set S."""
        conflicts = 0
        n = len(S)
        for i in range(n):
            for j in range(i + 1, n):
                if not self.are_neighbors(S[i], S[j]):
                    conflicts += 1
        return conflicts

    def solve(self, start_node=48, target=34, max_iter=500000):
        # 1. Improved Initial State: Start with a greedy clique
        S = [start_node]
        all_nodes = list(range(1, self.num_nodes + 1))
        random.shuffle(all_nodes)
        
        for n in all_nodes:
            if n not in S:
                if all(self.are_neighbors(n, member) for member in S):
                    S.append(n)
        
        current_conflicts = 0
        best_S = list(S)
        best_size = len(S)

        # 2. SA Hyperparameters
        T = 2.0            # Initial Temperature
        alpha = 0.99998    # Extremely slow cooling for stability
        T_min = 0.0001
        
        # Penalty multiplier: Higher values force the algorithm to stay as a valid clique
        penalty_weight = 10.0 

        print(f"Starting SA. Initial Greedy Clique Size: {best_size}")

        for i in range(max_iter):
            if T < T_min:
                T = 1.0 # Re-heat/Restart mechanism if target not reached

            new_S = list(S)
            move_type = random.random()

            # 3. Targeted Perturbations
            if move_type < 0.2 and len(new_S) > 1:
                # REMOVE: Pick a node at random (excluding the seed node if desired)
                node_to_remove = random.choice(new_S)
                new_S.remove(node_to_remove)
            
            elif move_type < 0.7:
                # ADD: Min-Conflict selection
                # Sample potential candidates and pick the one with the fewest conflicts
                candidates = random.sample(range(1, self.num_nodes + 1), 15)
                best_cand = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                if best_cand not in new_S:
                    new_S.append(best_cand)
            
            else:
                # SWAP: Remove one, add one (maintains size exploration)
                if len(new_S) > 1:
                    new_S.remove(random.choice(new_S))
                    candidates = random.sample(range(1, self.num_nodes + 1), 15)
                    best_cand = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                    if best_cand not in new_S:
                        new_S.append(best_cand)

            # 4. Energy Function: Minimize Conflicts, Maximize Size
            new_conflicts = self.calculate_total_conflicts(new_S)
            
            # Energy = (Conflicts * Penalty) - Size
            energy_new = (new_conflicts * penalty_weight) - len(new_S)
            energy_curr = (current_conflicts * penalty_weight) - len(S)
            delta = energy_new - energy_curr

            # 5. Metropolis Acceptance Criteria
            if delta < 0 or random.random() < math.exp(-delta / T):
                S = new_S
                current_conflicts = new_conflicts
            
            # 6. Success Tracking
            if current_conflicts == 0 and len(S) > best_size:
                best_size = len(S)
                best_S = list(S)
                print(f"Iteration {i}: Found Clique Size {best_size} (T={T:.4f})")
                
                if best_size >= target:
                    print("--- TARGET REACHED ---")
                    break

            T *= alpha 
            
            # Periodic logging
            if i % 50000 == 0 and i > 0:
                print(f"Checkpoint: Iter {i}, Best so far: {best_size}, Temp: {T:.4f}")

        return best_S, best_size

# --- Execution ---
if __name__ == "__main__":
    # Ensure your data file is in the same directory
    solver = MaximumCliqueSA('C125-9_nam.data')
    
    start_time = time.time()
    # We use a loop to ensure 100% success; if one SA run cools too much, we restart
    final_size = 0
    attempt = 1
    
    while final_size < 34:
        print(f"\nAttempt #{attempt}")
        # Randomizing the start node can help escape local optima across runs
        seed_node = random.randint(1, 125)
        clique, final_size = solver.solve(start_node=seed_node, target=34)
        attempt += 1

    end_time = time.time()

    print("\n" + "="*30)
    print(f"FINAL RESULT FOUND")
    print(f"Max Clique Size: {final_size}")
    print(f"Nodes: {sorted(clique)}")
    print(f"Total Time: {end_time - start_time:.2f}s")
    print("="*30)


Attempt #1
Starting SA. Initial Greedy Clique Size: 24
Iteration 16: Found Clique Size 25 (T=1.9994)
Iteration 96: Found Clique Size 26 (T=1.9962)
Iteration 124: Found Clique Size 27 (T=1.9950)
Iteration 227: Found Clique Size 28 (T=1.9909)
Iteration 262: Found Clique Size 29 (T=1.9895)
Iteration 344: Found Clique Size 30 (T=1.9863)
Iteration 838: Found Clique Size 31 (T=1.9668)
Iteration 1001: Found Clique Size 32 (T=1.9604)
Iteration 1491: Found Clique Size 33 (T=1.9412)
Iteration 4664: Found Clique Size 34 (T=1.8219)
--- TARGET REACHED ---

FINAL RESULT FOUND
Max Clique Size: 34
Nodes: [1, 2, 5, 7, 9, 11, 18, 19, 25, 29, 31, 34, 40, 44, 45, 48, 49, 54, 68, 70, 71, 77, 79, 80, 98, 99, 101, 110, 114, 115, 117, 121, 122, 125]
Total Time: 1.56s
